In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
import matplotlib.pyplot as plt

In [2]:
inTime = pd.read_csv('finalVariable_inTimeData3.csv')
outTime = pd.read_csv('finalVariable_outTimeData3.csv')

outTime.head(5)

,Unnamed: 0,FraudLabel,address-zip5-homephone_count30_date,address-zip5_0_count_address-zip5_30_count_Ave,address-zip5_0_count_address-zip5_7_count_Ave,address-zip5_count0_date,address-zip5_count180_date,address-zip5_count1_date,address-zip5_count30_date,address-zip5_count3_date,...,address_count1_date,address_count30_date,address_count3_date,address_count7_date,address_count90_date,name-dob_count180_date,name-dob_count30_date,name-dob_count90_date,ssn-dob_count180_date,ssn-dob_count30_date
0,833507,0,1,30.0,7.0,1,2,1,1,1,...,1,1,1,1,1,2,1,1,2,1
1,833508,0,1,30.0,7.0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,833509,0,2,15.0,7.0,1,2,1,2,1,...,1,2,1,1,2,2,2,2,2,2
3,833510,0,1,30.0,7.0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,833511,0,1,30.0,7.0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [3]:
len(inTime) + len(outTime)

967035

In [4]:
inTime.shape

(800542, 27)

In [5]:
inTime = inTime.drop(['Unnamed: 0'], axis=1)
outTime = outTime.drop(['Unnamed: 0'], axis=1)

In [6]:
inTimeX = inTime.drop('FraudLabel', axis=1)
inTimeY = inTime['FraudLabel']

In [7]:
pd.DataFrame(inTime['FraudLabel'].value_counts().values, inTime['FraudLabel'].value_counts().index )

,0
0,789000
1,11542


In [8]:
len(inTime[inTime['FraudLabel']==0])/len(inTime[inTime['FraudLabel']==1])    

68.35903656212095

In [9]:
# split intime data into train and test

In [10]:
from sklearn.model_selection import train_test_split
Y = inTimeY
X = inTimeX
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=20)
x_train = x_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)

x_train shape: (640433, 25)
x_test shape: (160109, 25)


In [11]:
x_OOT = outTime.drop('FraudLabel', axis=1) 
y_OOT = outTime['FraudLabel']

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
RF = RandomForestClassifier(n_estimators=500, criterion='entropy', class_weight = {0:1, 1:10})
RF = RF.fit(x_train, y_train)
y_pred = RF.predict(x_test)
print (metrics.classification_report(y_test, y_pred))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
def search_model(x_train, y_train, est, param_grid, n_jobs, cv):
    model = GridSearchCV(estimator  = est,
                                     param_grid = param_grid,
                                     scoring = 'f1_weighted',
                                     verbose = 10,
                                     n_jobs = n_jobs,
                                     iid = True,
                                     cv = cv)
    # Fit Grid Search Model
    model.fit(x_train, y_train)   
    return model

In [21]:
param_grid = {'n_estimators':[100,300,500],
             'criterion':['gini', 'entropy'],
             'class_weight': [{0:1, 1:5}, {0:1, 1:92}],
             'min_samples_leaf':[3,5],
             'max_depth':[10,20]}

RF = search_model(X.values
            , Y.values
            , RandomForestClassifier()
            , param_grid
            , -1
            , 5)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 10.6min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 14.1min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 18.1min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 22.7min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 30.2min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 35.1min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 39.3min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 46.1min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed: 55.7min
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed: 77.8min finished


In [24]:
print("Best score: %0.3f" % RF.best_score_)
print("Best parameters set:", RF.best_params_)
print("Scores:", RF.cv_results_) 

Best score: 0.990
Best parameters set: {'class_weight': {0: 1, 1: 5}, 'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 3, 'n_estimators': 300}
Scores: {'mean_fit_time': array([ 30.18990822, 105.78470016,  32.57753835, 100.93657627,
        50.89853368, 127.99932775,  41.91756024, 123.79531326,
        39.89776058, 128.94569216,  57.66583877, 153.77912469,
        58.70063806, 172.12603006,  53.80751615, 164.3550766 ,
        35.60086465, 142.4653563 ,  40.0131484 , 111.46442976,
        36.05064154, 128.53874702,  38.49912333, 144.18575015,
        60.84168887, 186.62262664,  64.05039973, 185.49440079,
        66.30080566, 247.70360918, 122.9385972 , 495.90073929]), 'std_fit_time': array([1.20238342e-01, 5.00792440e+00, 4.00736718e-01, 5.54833683e+00,
       6.56015159e+00, 2.87507065e+00, 8.34348279e-01, 2.67156933e+00,
       5.32081939e-01, 1.83124573e+01, 3.66232997e+00, 4.13502213e+00,
       6.08584423e+00, 7.00622542e+00, 3.60732362e+00, 1.06213048e+01,
       3.3428794

# Define FDR

In [12]:
def FDR(model,data_x, data_y):
    
    PredictedY = model.predict(data_x)        
    PredictedY = pd.Series(PredictedY)

    temp = pd.DataFrame(columns = ["PredictedY","RealY"])
    temp["PredictedY"] = PredictedY
    temp["RealY"] = data_y
    temp.sort_values(by = "PredictedY", ascending = False, inplace = True)

    ThreePercent = int(len(data_x) *0.03)
    ThreePercent

    FDR = temp["RealY"][:ThreePercent].sum() / data_y.sum()
    return FDR

### Test FDR on RF

In [13]:
Y = inTimeY
X = inTimeX
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=4)
x_train = x_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [21]:
from sklearn.ensemble import RandomForestClassifier

for n_estimator in [100, 200]:
    for  min_samples_leaf in [3,5,10,20]:
        for max_depth in [5, 10, 15, 20,30]:

            RF=RandomForestClassifier(n_estimators=n_estimator, criterion='entropy', 
                                      min_samples_leaf=min_samples_leaf, class_weight = {0:1, 1:5},
                                      max_depth = max_depth)
            RF.fit(x_train, y_train)

            Training = FDR(RF,x_train, y_train)
            Testing = FDR(RF, x_test, y_test)
            OOT = FDR(RF, x_OOT, y_OOT )
            print('n_estimator', n_estimator, 'min_samples_leaf', min_samples_leaf, 'max_depth', max_depth)
            print('Training', Training, 'Testing', Testing, 'OOT', OOT)
            print('------------------------------------------------')

n_estimator 100 min_samples_leaf 3 max_depth 5
Training 0.5083189282627485 Testing 0.5284339457567804 OOT 0.490360435875943
------------------------------------------------
n_estimator 100 min_samples_leaf 3 max_depth 10
Training 0.5203111495246326 Testing 0.5336832895888014 OOT 0.5004191114836547
------------------------------------------------
n_estimator 100 min_samples_leaf 3 max_depth 15
Training 0.5265773552290406 Testing 0.5402449693788276 OOT 0.5033528918692373
------------------------------------------------
n_estimator 100 min_samples_leaf 3 max_depth 20
Training 0.526361279170268 Testing 0.5406824146981627 OOT 0.5046102263202011
------------------------------------------------
n_estimator 100 min_samples_leaf 3 max_depth 30
Training 0.5264693171996543 Testing 0.5411198600174978 OOT 0.5054484492875104
------------------------------------------------
n_estimator 100 min_samples_leaf 5 max_depth 5
Training 0.5087510803802938 Testing 0.5301837270341208 OOT 0.49161777032690696
--

KeyboardInterrupt: 

In [17]:
'''Training = FDR(RF,x_train, y_train)
Testing = FDR(RF, x_test, y_test)
OOT = FDR(RF, x_OOT, y_OOT )
Training, Testing, OOT'''

(0.5262532411408816, 0.5415573053368329, 0.5058675607711651)

# Tree_Run 10 times 

In [42]:
#For Tree Model
FDRTable = pd.DataFrame(columns = ["Training","Testing","OOT"])
for i in range(10):

    Y = inTimeY
    X = inTimeX
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=i)

    from sklearn.tree import DecisionTreeClassifier
    tree = DecisionTreeClassifier(criterion = "gini",max_depth=5, min_samples_leaf=5) 
    tree.fit(x_train, y_train) 

    Training = FDR(tree,x_train, y_train)
    Testing = FDR(tree, x_test, y_test)
    OOT = FDR(tree, x_OOT, y_OOT )
    
    FDRTable = FDRTable.append({'Training' : Training , 'Testing' : Testing, "OOT": OOT} , ignore_index=True)

FDRTable = FDRTable.append({'Training' : FDRTable["Training"].mean(), "Testing":FDRTable['Testing'].mean(), "OOT": FDRTable["OOT"].mean()},ignore_index=True)
FDRTable

,Training,Testing,OOT
0,0.010057,0.005814,0.150838
1,0.007386,0.005236,0.128492
2,0.007496,0.004975,0.128492
3,0.007299,0.005464,0.189944
4,0.008633,0.005780,0.145251
5,0.005722,0.005917,0.111732
6,0.005944,0.005128,0.150838
7,0.007236,0.000000,0.162011
8,0.010204,0.005495,0.139665
9,0.002963,0.000000,0.134078


In [315]:
FDRTable.to_excel('Tree_FDRTable.xls')

# RF_Run 10 times 

In [25]:
FDRTable = pd.DataFrame(columns = ["Training","Testing","OOT"])
for i in range(10):   
    
    Y = inTimeY
    X = inTimeX
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=i)
    x_train = x_train.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    x_test = x_test.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)
    
    from sklearn.ensemble import RandomForestClassifier
    RF=RandomForestClassifier(n_estimators=300, criterion='gini', min_samples_leaf=3, class_weight = {0:1, 1:5}, max_depth = 20)
    RF.fit(x_train, y_train)
        
    Training = FDR(RF,x_train, y_train)
    Testing = FDR(RF, x_test, y_test)
    OOT = FDR(RF, x_OOT, y_OOT )
    
    FDRTable = FDRTable.append({'Training' : Training , 'Testing' : Testing, "OOT": OOT} , ignore_index=True)

FDRTable = FDRTable.append({'Training' : FDRTable["Training"].mean(), "Testing":FDRTable['Testing'].mean(), "OOT": FDRTable["OOT"].mean()},ignore_index=True)
FDRTable

,Training,Testing,OOT
0,0.844828,0.575581,0.173184
1,0.838996,0.523560,0.178771
2,0.850075,0.562189,0.173184
3,0.830657,0.540984,0.184358
4,0.854676,0.549133,0.195531
5,0.862661,0.550296,0.184358
6,0.841010,0.517949,0.173184
7,0.849493,0.553672,0.184358
8,0.833819,0.615385,0.178771
9,0.848889,0.575130,0.184358


In [25]:
FDRTable.to_excel('RF_FDRTable.xls')